# Computation of layer by layer containment corrections




## Setup

In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools


In [17]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v87t'
version_V8 = version
version_V9 = 'v112'
version_V10 = 'v106'


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version_V8, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version_V8, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_calibs', 'PU0', version_V9, 'ele-V9'),
    Sample('ele_flat2to100_PU200_calibs', 'PU200', version_V9, 'ele-V9')
    ]

samples_ele_V10 = [
    Sample('ele_flat2to100_PU0_calibs', 'PU0', version_V10, 'ele-V10'),
    Sample('ele_flat2to100_PU200_tps', 'PU200', version_V10, 'ele-V10')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0_tps', 'PU0', version, 'photon'),
#     Sample('photon_flat8to150_PU200_tps', 'PU200', version, 'photon')
    ]

samples_photons_V8 = [
    Sample('photon_flat8to150_PU0', 'PU0', version_V8, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version_V8, 'photon')
    ]

samples_photons_V9 = [
    Sample('ele_flat2to100_PU0_calibs', 'PU0', version_V9, 'photons-V9'),
    # FIXME
#     Sample('photon_flat8to150_PU0_calibs', 'PU0', version_V9, 'photons-V9'),
#     Sample('photon_flat8to150_PU200_tps', 'PU200', version_V9, 'photons-V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V9')
    ]


In [29]:
# %load settings.py
# %load settings.py
# === samples =====================================================
import python.selections as selections
import python.collections as collections

import pprint


samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
samples += samples_photons_V9
samples += samples_ele_V9

for smp in samples:
    smp.open_file()


sample = 'ele-V9'

do_rate = False
do_eff = False
do_reso = False
do_calib = True

# === TP ==========================================================
tps = ['HMvDR',
       'HMvDRshapeDr']



# === Load the Histo Primitives ====================================
histo_primitives = pd.DataFrame()

if do_rate:
    from python.plotters import rate_plotters, eg_rate_plotters

    for plotter in rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in eg_rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
if do_eff or do_reso:
    from python.plotters import eg_genmatched_plotters, track_genmatched_plotters
    for plotter in eg_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in track_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
if do_calib:
    from python.plotters import tp_calib_plotters

    for plotter in tp_calib_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)


# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.data == tp].data_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.data == tp].gen_sel.unique().tolist()

#  ==== labels ===============================================
tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]



import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


# print selections.eg_rate_selections


--- TPs: 
['HMvDR', 'HMvDRshapeDr']
--- TP selections:
{   'HMvDR': ['all', 'Em'], 'HMvDRshapeDr': ['all', 'Em']}
--- GEN selections:
{   'HMvDR': [   'GEN',
                 'GENEtaD',
                 'GENEtaBC',
                 'GENEtaBCD',
                 'GENEtaBCPt15',
                 'GENEtaBCPt20',
                 'GENEtaBCPt30',
                 'GENEtaBCPt40'],
    'HMvDRshapeDr': [   'GEN',
                        'GENEtaD',
                        'GENEtaBC',
                        'GENEtaBCD',
                        'GENEtaBCPt15',
                        'GENEtaBCPt20',
                        'GENEtaBCPt30',
                        'GENEtaBCPt40']}


In [25]:
tp_labels


{'HMvDR': 'HM+dR(layer) Cl3d', 'HMvDRshapeDr': 'HM #Delta#rho < 0.015'}

In [26]:
tp_selection_labels

{'Em': 'EGId', 'all': ''}

In [27]:
gen_selection_labels

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCPt15': '1.52 < |#eta^{GEN}| <= 2.4, p_{T}^{GEN}>=15GeV',
 'GENEtaBCPt20': '1.52 < |#eta^{GEN}| <= 2.4, p_{T}^{GEN}>=20GeV',
 'GENEtaBCPt30': '1.52 < |#eta^{GEN}| <= 2.4, p_{T}^{GEN}>=30GeV',
 'GENEtaBCPt40': '1.52 < |#eta^{GEN}| <= 2.4, p_{T}^{GEN}>=40GeV',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8'}

In [30]:
samples

[<Sample ../plots1/histos_ele_flat2to100_PU0_calibs_v112.root, photons-V9>,
 <Sample ../plots1/histos_ele_flat2to100_PU0_calibs_v112.root, ele-V9>,
 <Sample ../plots1/histos_ele_flat2to100_PU200_calibs_v112.root, ele-V9>]

In [31]:
files

{}

In [32]:
class EnergyRes(histos.ResoHistos):
    def __init__(self,  name, root_file=None, debug=False):
        self.name = name
        self.h_energyResVenergy = ROOT.TH2F(self.name + '_energyResVenergy', '', 100,0,1000, 200, -100, 100)
        self.h_energyResp = ROOT.TH1F(self.name+'_energyResp; E^{L1}/E^{GEN}', '', 200, 0, 2)
        self.h_energyRes = ROOT.TH1F(self.name+'_energyRes; E^{L1}-E^{GEN}/E^{GEN}', '', 200, -10, 10)

        
        self.h_lCalibEnergyResVenergy = ROOT.TH2F(self.name+'_lCalibEnergyResVenergy', '', 100,0,1000, 200, -100, 100)
        self.h_lCalibEnergyResp = ROOT.TH1F(self.name + '_lCalibEnergyResp; E^{L1}/E^{GEN}', '', 200, 0, 2)

        self.h_etaEnergyResVenergy = ROOT.TH2F(self.name + '_etaEnergyResVenergy', '', 100,0,1000, 200, -100, 100)
        self.h_etaEnergyResp = ROOT.TH1F(self.name + '_etaEnergyResp; E^{L1}/E^{GEN}', '', 200, 0, 2)

        self.h_etalCalibEnergyResVenergy = ROOT.TH2F(self.name + '_etalCalibEnergyResVenergy', '', 100,0,1000, 200, -100, 100)
        self.h_etalCalibEnergyResp = ROOT.TH1F(self.name + '_etalCalibEnergyResp; E^{L1}/E^{GEN}', '', 200, 0, 2)
    
        self.h_energyRes = ROOT.TH1F(self.name+'h_energyRes; (E^{L1}-E^{GEN})/E^{GEN}', '', 200, -2, 2)
        self.h_etaEnergyRes = ROOT.TH1F(self.name+'h_etaEnergyRes; (E^{L1}-E^{GEN})/E^{GEN}', '', 200, -2, 2)

        
        histos.BaseResoHistos.__init__(self, name, root_file)
    
    def fill(self, tree, expression):
        hname = '{}_energyResVenergy'.format(self.name)
        draw_expression = '{}>>{}(100,0,1000, 200, -100, 100)'.format(expression, hname)
        tree.Draw(draw_expression,"","COL")
        self.h_energyResVenergy = ROOT.gPad.GetPrimitive(hname)
        
        
    def fill_df(self, df):
        rnp.fill_hist(self.h_energyResVenergy, df[['gen_energy', 'uncalib_res']])
        rnp.fill_hist(self.h_lCalibEnergyResVenergy, df[['gen_energy', 'lcalib_res']])
        rnp.fill_hist(self.h_etaEnergyResVenergy, df[['gen_energy', 'eta_uncalib_res']])
        rnp.fill_hist(self.h_etalCalibEnergyResVenergy, df[['gen_energy', 'eta_lcalib_res']])
        rnp.fill_hist(self.h_energyRes, df.uncalib_res/df.gen_energy)

        rnp.fill_hist(self.h_energyResp, df.uncalib_energy_nl0/df.gen_energy)
        rnp.fill_hist(self.h_lCalibEnergyResp, df.lcalib_energy/df.gen_energy)
        rnp.fill_hist(self.h_etaEnergyResp, df.eta_uncalib_energy_nl0/df.gen_energy)
        rnp.fill_hist(self.h_etalCalibEnergyResp, df.eta_lcalib_energy/df.gen_energy)
        rnp.fill_hist(self.h_energyRes, (df.uncalib_res)/df.gen_energy)
        rnp.fill_hist(self.h_etaEnergyRes, (df.eta_uncalib_res)/df.gen_energy)
        
        
class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo


## Computation of the values

In [33]:
calib_table = {}
# calib_table['HMvDRfixedCalib'] = [4.5, 0.94, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.91, 1.06, 1.07, 1.37, 1.47]
# calib_table['HMvDRcylind10Calib'] = [4.5, 0.94, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.91, 1.06, 1.07, 1.37, 1.47]
# calib_table['HMvDRcylind5Calib'] = [4.91, 0.94, 1.09, 1.05, 0.92, 1.05, 1.11, 0.94, 1.02, 0.92, 1.1, 1.04, 1.34, 1.6]
# calib_table['HMvDRcylind2p5Calib'] = [8.49, 0.97, 1.19, 1.18, 1.04, 1.1, 1.25, 1.05, 1.14, 0.97, 1.17, 1.34, 1.3, 1.67]
#                                      [7.86, 1.1, 1.23, 1.0, 1.18, 1.23, 0.99, 1.15, 1.11, 1.16, 0.98, 1.4, 1.47, 1.45]

# # v89
# calib_table['HMvDRfixedCalib'] = [4.52, 0.93, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.9, 1.07, 1.06, 1.36, 1.47]
# calib_table['HMvDRcylind10Calib'] = [4.53, 0.88, 1.14, 1.0, 0.92, 1.06, 1.07, 0.89, 1.06, 0.83, 1.07, 1.13, 1.34, 1.49]
# calib_table['HMvDRcylind5Calib'] = [4.98, 1.03, 0.98, 1.12, 0.91, 1.05, 1.14, 0.87, 1.02, 0.97, 1.13, 1.01, 1.43, 1.4]
# calib_table['HMvDRcylind2p5Calib'] = [7.86, 1.1, 1.23, 1.0, 1.18, 1.23, 0.99, 1.15, 1.11, 1.16, 0.98, 1.4, 1.47, 1.45]
# calib_table['HMvDRshapeCalib'] = [1]*28

# # v93
# calib_table['HMvDRCalib'] = [6.38, 0.98, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.81, 1.0, 0.9, 1.08, 1.5, 1.81]
# calib_table['HMvDRcylind10Calib'] = [6.39, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
# calib_table['HMvDRcylind5Calib'] = [6.92, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
# calib_table['HMvDRcylind2p5Calib'] = [12.35, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
# calib_table['HMvDRshapeCalib'] = [8.43, 1.69, 1.1, 1.2, 0.97, 0.97, 1.13, 1.03, 0.82, 1.0, 0.89, 1.08, 1.55, 1.82]
# calib_table['HMvDRshapeDrCalib'] = [1.]*14

# # v96
# calib_table['HMvDRCalib'] = [6.38, 0.98, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.81, 1.0, 0.9, 1.08, 1.5, 1.81]
# calib_table['HMvDRcylind10Calib'] = [6.39, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
# calib_table['HMvDRcylind5Calib'] = [6.92, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
# calib_table['HMvDRcylind2p5Calib'] = [12.35, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
# calib_table['HMvDRshapeCalib'] = [7.17, 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
# calib_table['HMvDRshapeDrCalib'] = [6.96, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.05, 0.83, 1.03, 0.91, 1.07, 1.51, 1.89]

# v96bis
# calib_table['HMvDRCalib'] = [7.16, 2.03, 1.06, 0.75, 1.24, 0.96, 0.83, 1.15, 1.06, 0.87, 0.86, 0.93, 1.24, 2.59]
# calib_table['HMvDRcylind10Calib'] = [1., 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
# calib_table['HMvDRcylind5Calib'] = [1., 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
# calib_table['HMvDRcylind2p5Calib'] = [1., 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
# calib_table['HMvDRshapeCalib'] = [1., 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
# calib_table['HMvDRshapeDrCalib'] = [7.43, 2.14, 1.06, 0.84, 1.14, 1.0, 0.99, 1.0, 1.07, 0.95, 0.9, 0.91, 1.19, 2.71]


calib_table['HMvDRCalib'] = [0.0, 2.64, 0.95, 0.76, 1.19, 0.98, 0.85, 1.15, 1.04, 0.86, 0.87, 0.93, 1.27, 2.56]
calib_table['HMvDRcylind10Calib'] = [1., 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
calib_table['HMvDRcylind5Calib'] = [1., 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
calib_table['HMvDRcylind2p5Calib'] = [1., 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
calib_table['HMvDRshapeCalib'] = [1., 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
# calib_table['HMvDRshapeDr'] = [0., 1.6144949, 0.92495334, 1.0820811, 0.9753549, 0.9742881, 1.0634482, 1.0599478, 0.9376349, 0.92587173, 0.8003076, 1.0417082, 1.7032381, 2.]
# calib_table['HMvDRshapeDrCalib'] = [0., 2.0, 1.35, 0.82, 1.03, 1.01, 1.01, 1.03, 1.07, 0.91, 0.87, 1.08, 1.42, 1.99]
# calib_table['HMvDRshapeDrCalib'] = [2.0, 1.47, 0.96, 1.09, 0.95, 1.09, 1.02, 0.99, 0.94, 1.1, 1.21, 0.96, 1.33, 1.93]
calib_table['HMvDRshapeDrCalib'] =  [0.] + [1.]*13

# calib_table['HMvDR'] = [0.] + [1.]*13
# calib_table['HMvDRcylind10'] =  [0.] + [1.]*13
# calib_table['HMvDRcylind5'] =  [0.] + [1.]*13
# calib_table['HMvDRcylind2p5'] =  [0.] + [1.]*13
# calib_table['HMvDRshape'] =  [0.] + [1.]*13
# calib_table['HMvDRshapeDr'] =  [0.] + [1.]*13


# v102 ele-V10
calib_table_V10 = {}
calib_table_V10['HMvDRCalib'] = [8.04, 1.61, 1.08, 0.8, 1.23, 1.04, 0.91, 1.16, 1.05, 1.04, 0.84, 1.05, 1.36, 2.48]
calib_table_V10['HMvDRcylind10Calib'] = [1., 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
calib_table_V10['HMvDRcylind5Calib'] = [8.6, 1.63, 1.09, 0.92, 1.12, 1.1, 1.05, 1.03, 1.09, 1.08, 0.94, 0.97, 1.42, 2.59]
calib_table_V10['HMvDRcylind2p5Calib'] = [1., 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
calib_table_V10['HMvDRshapeCalib'] = [1., 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
calib_table_V10['HMvDRshapeDrCalib'] = [8.65, 1.63, 1.09, 0.93, 1.11, 1.1, 1.05, 1.03, 1.07, 1.09, 0.94, 0.98, 1.36, 2.65]

print calib_table


calib_table_by_sample = {}
calib_table_by_sample['ele-V10'] = calib_table_V10
calib_table_by_sample['ele-V9'] = calib_table
calib_table_by_sample['photons-V10'] = calib_table_V10
calib_table_by_sample['photons-V9'] = calib_table

{'HMvDRCalib': [0.0, 2.64, 0.95, 0.76, 1.19, 0.98, 0.85, 1.15, 1.04, 0.86, 0.87, 0.93, 1.27, 2.56], 'HMvDRcylind5Calib': [1.0, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89], 'HMvDRcylind2p5Calib': [1.0, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15], 'HMvDRshapeCalib': [1.0, 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84], 'HMvDRshapeDrCalib': [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'HMvDRcylind10Calib': [1.0, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]}


In [34]:
tps

['HMvDR', 'HMvDRshapeDr']

### read the calib trees

In [35]:
samples = []
samples += samples_photons_V9
samples += samples_ele_V9
# samples += samples_ele_V10


import array
import numpy as np 

tree_data = pd.DataFrame()

for smp in samples:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
                tree_name = '{}_{}_{}_calib'.format(tp, tp_sel, gen_sel)
                if smp.type == 'ele-V9' and tp == "HMvDRCalib":
                    tree_name = '{}_{}_{}_calib'.format('HMvDR', tp_sel, gen_sel)
                # get the tree
                tree = smp.histo_file.Get(tree_name)
                
                # read the branches
                df = pd.DataFrame()

                cluster_pt = np.array([list(aset)[0] for aset in rnp.tree2array(tree, branches=['pt'], cache_size=400000000)])
                cluster_eta = np.array([list(aset)[0] for aset in rnp.tree2array(tree, branches=['eta'], cache_size=400000000)])
                branches = ['e1', 'e3','e5','e7','e9','e11','e13','e15','e17','e19','e21','e23','e25','e27']

                cluster_layer_energies = np.array([np.array(list(aset)) for aset in rnp.tree2array(tree, branches=branches, cache_size=400000000)])
                gen_energy = np.array(rnp.tree2array(tree, branches='Egen', cache_size=400000000))
                
                # fill the array

                df['cl_pt'] = cluster_pt
                df['cl_layer_energies'] = cluster_layer_energies.tolist()
                df['cl_eta'] = cluster_eta
                df['cl_abseta'] = np.abs(df.cl_eta)
                df['gen_energy'] = gen_energy
                # FIXME: approx
                df['gen_pt'] = df.gen_energy/np.cosh(df.cl_eta)
        
                
                df['uncalib_energy'] = df.cl_layer_energies.apply(np.sum)
                df['uncalib_energy_nl0'] = df.cl_layer_energies.apply(lambda x: np.dot(x, [0]+[1.]*13))

                df.loc[:, 'smp'] = smp.type
                df.loc[:, 'pu'] = smp.label
                df.loc[:, 'tp'] = tp
                df.loc[:, 'gen_sel'] = gen_sel
                df.loc[:, 'tp_sel'] = tp_sel

                tree_data = tree_data.append(df, ignore_index=True)



------ SAMPLE: photons-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD


In [36]:
tree_data.iloc[:10]

,cl_pt,cl_layer_energies,cl_eta,cl_abseta,gen_energy,gen_pt,uncalib_energy,uncalib_energy_nl0,smp,pu,tp,gen_sel,tp_sel
0,5.523190,"[0.0952823981643, 1.04982435703, 3.19946479797...",2.267544,2.267544,31.082659,6.369903,26.953290,26.858007,photons-V9,PU0,HMvDR,GENEtaBC,Em
1,23.623173,"[0.0777303799987, 2.99949836731, 5.25537061691...",-1.894929,1.894929,91.581436,26.926203,79.695659,79.617929,photons-V9,PU0,HMvDR,GENEtaBC,Em
2,31.489098,"[0.240713432431, 5.29911279678, 13.7976922989,...",-2.209277,2.209277,148.637558,32.246235,145.166468,144.925755,photons-V9,PU0,HMvDR,GENEtaBC,Em
3,87.224854,"[0.0, 2.34960699081, 14.6975412369, 44.3800735...",2.386547,2.386547,486.873138,88.782135,478.407465,478.407465,photons-V9,PU0,HMvDR,GENEtaBC,Em
4,78.769875,"[0.190564796329, 10.1607999802, 32.2133598328,...",1.875911,1.875911,265.762390,79.569847,263.115332,262.924767,photons-V9,PU0,HMvDR,GENEtaBC,Em
5,12.489261,"[0.135401308537, 3.14947319031, 7.13630628586,...",-1.924889,1.924889,70.597046,20.169779,43.715611,43.580210,photons-V9,PU0,HMvDR,GENEtaBC,Em
6,75.284500,"[0.43127822876, 10.0483188629, 26.3705883026, ...",-2.328539,2.328539,486.232880,93.863968,388.616345,388.185067,photons-V9,PU0,HMvDR,GENEtaBC,Em
7,55.441338,"[0.305906653404, 11.0419025421, 29.4075813293,...",-1.801000,1.801000,182.924820,58.810390,172.433364,172.127457,photons-V9,PU0,HMvDR,GENEtaBC,Em
8,52.842434,"[0.0, 4.99916362762, 24.620880127, 46.80467224...",2.314880,2.314880,263.317505,51.517307,270.129819,270.129819,photons-V9,PU0,HMvDR,GENEtaBC,Em
9,46.731613,"[0.0, 3.44317388535, 18.8968391418, 39.8308372...",2.171803,2.171803,214.692398,48.310295,207.677757,207.677757,photons-V9,PU0,HMvDR,GENEtaBC,Em


### compute the layer-by-layer corrections

In [37]:
# compute the layer calibrations on photons

calib_df = pd.DataFrame()
from sklearn.linear_model import LinearRegression
from scipy.optimize import least_squares
from scipy.optimize import lsq_linear

for smp in samples_photons_V9:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]
                
                
#                 res = least_squares(fun=compute_residual, 
#                                     x0=np.array([1.]*14), 
#                                     args=(layer_energy_list, target_energy),
#                                     bounds=(np.array([0.5]*14), np.array([2.]*14)))
#                 print '   - least. sq bound: {}'.format([float("{0:0.2f}".format(i)) for i in res.x ])
#                 print sel_df.cl_layer_energies.tolist()
                lsqbregr=lsq_linear(np.array(sel_df.cl_layer_energies.tolist()), 
                                    sel_df.gen_energy.to_numpy(), 
                                    bounds = (np.array([0.5]*14), np.array([2.]*14)), 
                                    method='bvls', lsmr_tol='auto', verbose=1)
                weights = lsqbregr.x
                print '   - linear. lsq bound: {}'.format([float("{0:0.2f}".format(i)) for i in lsqbregr.x ])

                df = pd.DataFrame()
                df['layer_calib'] = [lsqbregr.x]
                df.loc[:, 'smp'] = smp.type
                df.loc[:, 'pu'] = smp.label
                df.loc[:, 'tp'] = tp
                df.loc[:, 'gen_sel'] = gen_sel
                df.loc[:, 'tp_sel'] = tp_sel

                calib_df = calib_df.append(df, ignore_index=True)

                


------ SAMPLE: photons-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
The relative change of the cost function is less than `tol`.
Number of iterations 3, initial cost 3.5273e+06, final cost 3.4674e+06, first-order optimality 4.78e-07.
   - linear. lsq bound: [2.0, 2.0, 1.31, 0.75, 1.12, 0.96, 1.06, 1.02, 0.97, 0.88, 0.97, 1.0, 1.3, 2.0]
TP: HMvDR sel: Em, gen_sel: GENEtaD
The relative change of the cost function is less than `tol`.
Number of iterations 5, initial cost 2.2872e+07, final cost 2.2478e+07, first-order optimality 1.38e-08.
   - linear. lsq bound: [2.0, 2.0, 1.05, 0.92, 1.2, 1.67, 0.65, 0.88, 0.86, 1.31, 0.76, 0.82, 2.0, 2.0]
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
The relative change of the cost function is less than `tol`.
Number of iterations 3, initial cost 2.7986e+07, final cost 2.7825e+07, first-order optimality 1.04e-06.
   - linear. lsq bound: [2.0, 2.0, 0.94, 0.88, 1.12, 1.36, 0.97, 0.88, 0.9, 1.17, 0.97, 0.98, 1.61, 2.0]
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaB

In [38]:
calib_df.iloc[:10]

,layer_calib,smp,pu,tp,gen_sel,tp_sel
0,"[2.0, 2.0, 1.31060586365, 0.750712522915, 1.11...",photons-V9,PU0,HMvDR,GENEtaBC,Em
1,"[2.0, 2.0, 1.04629579191, 0.920827106595, 1.20...",photons-V9,PU0,HMvDR,GENEtaD,Em
2,"[2.0, 2.0, 0.936643554191, 0.876934988126, 1.1...",photons-V9,PU0,HMvDR,GENEtaBCD,Em
3,"[2.0, 2.0, 1.3875111331, 0.766071128339, 1.094...",photons-V9,PU0,HMvDRshapeDr,GENEtaBC,Em
4,"[2.0, 2.0, 1.1063045314, 0.90686529152, 1.2208...",photons-V9,PU0,HMvDRshapeDr,GENEtaD,Em
5,"[2.0, 2.0, 1.03835423016, 0.852750906825, 1.15...",photons-V9,PU0,HMvDRshapeDr,GENEtaBCD,Em


### Apply layer-by-layer corrections

In [39]:
# apply layer by layer
calib_eta = 'GENEtaBC'
calib_sample = 'photons-V9'
calib_pu = 'PU0'

for smp in samples:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]
                
                calib = calib_df[(calib_df.smp == calib_sample) &
                                 (calib_df.pu == calib_pu) &
                                   (calib_df.tp == tp) &
                                   (calib_df.tp_sel == tp_sel) &
                                   (calib_df.gen_sel == calib_eta)]
                calib_values = calib.layer_calib.values[0]
                # we set to 0 layer 0
                calib_values[0] = 0
#                 print calib_values
                tree_data.loc[sel_df.index, 'lcalib_energy'] = sel_df.cl_layer_energies.apply(lambda x: np.dot(x, calib_values))
                


------ SAMPLE: photons-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD


In [40]:
tree_data[tree_data.smp == 'ele-V9'].iloc[:10]

,cl_pt,cl_layer_energies,cl_eta,cl_abseta,gen_energy,gen_pt,uncalib_energy,uncalib_energy_nl0,smp,pu,tp,gen_sel,tp_sel,lcalib_energy
147734,5.523190,"[0.0952823981643, 1.04982435703, 3.19946479797...",2.267544,2.267544,31.082659,6.369903,26.953290,26.858007,ele-V9,PU0,HMvDR,GENEtaBC,Em,28.542836
147735,23.623173,"[0.0777303799987, 2.99949836731, 5.25537061691...",-1.894929,1.894929,91.581436,26.926203,79.695659,79.617929,ele-V9,PU0,HMvDR,GENEtaBC,Em,83.094493
147736,31.489098,"[0.240713432431, 5.29911279678, 13.7976922989,...",-2.209277,2.209277,148.637558,32.246235,145.166468,144.925755,ele-V9,PU0,HMvDR,GENEtaBC,Em,152.521209
147737,87.224854,"[0.0, 2.34960699081, 14.6975412369, 44.3800735...",2.386547,2.386547,486.873138,88.782135,478.407465,478.407465,ele-V9,PU0,HMvDR,GENEtaBC,Em,483.117823
147738,78.769875,"[0.190564796329, 10.1607999802, 32.2133598328,...",1.875911,1.875911,265.762390,79.569847,263.115332,262.924767,ele-V9,PU0,HMvDR,GENEtaBC,Em,274.669457
147739,12.489261,"[0.135401308537, 3.14947319031, 7.13630628586,...",-1.924889,1.924889,70.597046,20.169779,43.715611,43.580210,ele-V9,PU0,HMvDR,GENEtaBC,Em,47.852841
147740,75.284500,"[0.43127822876, 10.0483188629, 26.3705883026, ...",-2.328539,2.328539,486.232880,93.863968,388.616345,388.185067,ele-V9,PU0,HMvDR,GENEtaBC,Em,407.270809
147741,55.441338,"[0.305906653404, 11.0419025421, 29.4075813293,...",-1.801000,1.801000,182.924820,58.810390,172.433364,172.127457,ele-V9,PU0,HMvDR,GENEtaBC,Em,188.483516
147742,52.842434,"[0.0, 4.99916362762, 24.620880127, 46.80467224...",2.314880,2.314880,263.317505,51.517307,270.129819,270.129819,ele-V9,PU0,HMvDR,GENEtaBC,Em,276.942120
147743,46.731613,"[0.0, 3.44317388535, 18.8968391418, 39.8308372...",2.171803,2.171803,214.692398,48.310295,207.677757,207.677757,ele-V9,PU0,HMvDR,GENEtaBC,Em,211.920359


### compute eta scale correction @ PU200

In [41]:
# compute the layer calibrations on photons

eta_calib_df = pd.DataFrame()
from sklearn.linear_model import LinearRegression
from scipy.optimize import least_squares
from scipy.optimize import lsq_linear

for smp in samples:
    if smp.label != 'PU200':
        continue

    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]

                df = pd.DataFrame(columns=['eta_calib_0', 'eta_calib_1', 'smp', 'pu', 'tp', 'gen_sel', 'tp_sel'])

                # uncalib eta correction

                X = sel_df.cl_abseta.to_numpy().reshape(-1, 1)
                Y = sel_df.gen_energy - sel_df.uncalib_energy_nl0.to_numpy()
                corr_eta = LinearRegression(fit_intercept=True).fit(X, Y)
                print corr_eta.coef_[0], corr_eta.intercept_
    
                df.loc[0, 'eta_calib_0'] = (corr_eta.coef_[0], corr_eta.intercept_)

#                 Y = sel_df.gen_energ


------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
-16.7822 37.8003
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
17.3006 -26.6416
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
-12.568 34.1704
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD
20.7719 -28.8493


In [42]:
# compute the layer calibrations on photons

eta_calib_df = pd.DataFrame()
from sklearn.linear_model import LinearRegression
from scipy.optimize import least_squares
from scipy.optimize import lsq_linear

for smp in samples:
    if smp.label != 'PU200':
        continue

    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]

                df = pd.DataFrame(columns=['eta_calib_0', 'eta_calib_1', 'smp', 'pu', 'tp', 'gen_sel', 'tp_sel'])

                # uncalib eta correction

                X = sel_df.cl_abseta.to_numpy().reshape(-1, 1)
                Y = sel_df.gen_energy - sel_df.uncalib_energy_nl0.to_numpy()
                corr_eta = LinearRegression(fit_intercept=True).fit(X, Y)
                print corr_eta.coef_[0], corr_eta.intercept_
    
                df.loc[0, 'eta_calib_0'] = (corr_eta.coef_[0], corr_eta.intercept_)

                if False:
                    Y = sel_df.gen_energy - sel_df.lcalib_energy.to_numpy()
                    corr_eta = LinearRegression(fit_intercept=True).fit(X, Y)
                    print corr_eta.coef_[0], corr_eta.intercept_

                    df.loc[0, 'eta_calib_1'] = (corr_eta.coef_[0], corr_eta.intercept_)

                    Y = sel_df.gen_pt - sel_df.lcalib_energy.to_numpy()/np.cosh(sel_df.cl_eta)
                    corr_eta = LinearRegression(fit_intercept=True).fit(X, Y)
                    print corr_eta.coef_[0], corr_eta.intercept_
                else:
                    df.loc[0, 'eta_calib_1'] = (0, 0)

                df.loc[:, 'smp'] = smp.type
                df.loc[:, 'pu'] = smp.label
                df.loc[:, 'tp'] = tp
                df.loc[:, 'gen_sel'] = gen_sel
                df.loc[:, 'tp_sel'] = tp_sel

                eta_calib_df = eta_calib_df.append(df, ignore_index=True)

                


------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
-16.7822 37.8003
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
17.3006 -26.6416
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
-12.568 34.1704
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD
20.7719 -28.8493


In [43]:
eta_calib_df

,eta_calib_0,eta_calib_1,smp,pu,tp,gen_sel,tp_sel
0,"(-16.7822, 37.8003)","(0, 0)",ele-V9,PU200,HMvDR,GENEtaBC,Em
1,"(17.3006, -26.6416)","(0, 0)",ele-V9,PU200,HMvDR,GENEtaBCD,Em
2,"(-12.568, 34.1704)","(0, 0)",ele-V9,PU200,HMvDRshapeDr,GENEtaBC,Em
3,"(20.7719, -28.8493)","(0, 0)",ele-V9,PU200,HMvDRshapeDr,GENEtaBCD,Em


In [44]:
# apply eta dependent corrections
eta_calib_eta = 'GENEtaBC'
eta_calib_sample = 'ele-V9'
eta_calib_pu = 'PU200'

for smp in samples_ele_V9:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)

                sel_df = tree_data[(tree_data.smp == smp.type) &
                                   (tree_data.pu == smp.label) &
                                   (tree_data.tp == tp) &
                                   (tree_data.tp_sel == tp_sel) &
                                   (tree_data.gen_sel == gen_sel)]
                
                calib = eta_calib_df[(eta_calib_df.smp == eta_calib_sample) &
                                 (eta_calib_df.pu == eta_calib_pu) &
                                   (eta_calib_df.tp == tp) &
                                   (eta_calib_df.tp_sel == tp_sel) &
                                   (eta_calib_df.gen_sel == eta_calib_eta)]
                calib_values = calib.eta_calib_0.values[0]
                print calib_values
                tree_data.loc[sel_df.index, 'eta_uncalib_energy_nl0'] = sel_df.uncalib_energy_nl0+calib_values[1]+sel_df.cl_abseta*calib_values[0]
                calib_values = calib.eta_calib_1.values[0]
                print calib_values
                tree_data.loc[sel_df.index, 'eta_lcalib_energy'] = sel_df.lcalib_energy+calib_values[1]+sel_df.cl_abseta*calib_values[0]

                # we set to 0 layer 0
                # calib_values[0] = 0
#                 print calib_values
                # tree_data.loc[sel_df.index, 'eta_lcalib_energy'] = sel_df.cl_layer_energies.apply(lambda x: np.dot(x, calib_values))
                


------ SAMPLE: ele-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
(-16.782204, 37.800282)
(0, 0)
TP: HMvDR sel: Em, gen_sel: GENEtaD
(-16.782204, 37.800282)
(0, 0)
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
(-16.782204, 37.800282)
(0, 0)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
(-12.56804, 34.170414)
(0, 0)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
(-12.56804, 34.170414)
(0, 0)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD
(-12.56804, 34.170414)
(0, 0)

------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
(-16.782204, 37.800282)
(0, 0)
TP: HMvDR sel: Em, gen_sel: GENEtaD
(-16.782204, 37.800282)
(0, 0)
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
(-16.782204, 37.800282)
(0, 0)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
(-12.56804, 34.170414)
(0, 0)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
(-12.56804, 34.170414)
(0, 0)
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD
(-12.56804, 34.170414)
(0, 0)


In [45]:
#  1: 34.2000 2: -12.5900 eta: 2.1211 delta: 59.9530
#  1: 34.2000 2: -12.5900 eta: 2.1031 delta: 59.3363
#  1: 34.2000 2: -12.5900 eta: 2.1087 delta: 59.5279

In [46]:
tp = 'HMvDRshapeDr'
tp_sel = 'Em'
eta = 1.5653

calib = eta_calib_df[(eta_calib_df.smp == eta_calib_sample) &
                                 (eta_calib_df.pu == eta_calib_pu) &
                                   (eta_calib_df.tp == tp) &
                                   (eta_calib_df.tp_sel == tp_sel) &
                                   (eta_calib_df.gen_sel == eta_calib_eta)]
calib_values = calib.eta_calib_0.values[0]
print calib_values

print calib_values[1]+eta*calib_values[0]

(-12.56804, 34.170414)
14.4976611247


In [47]:
tree_data[tree_data.smp == 'ele-V9'].iloc[:10]

,cl_pt,cl_layer_energies,cl_eta,cl_abseta,gen_energy,gen_pt,uncalib_energy,uncalib_energy_nl0,smp,pu,tp,gen_sel,tp_sel,lcalib_energy,eta_uncalib_energy_nl0,eta_lcalib_energy
147734,5.523190,"[0.0952823981643, 1.04982435703, 3.19946479797...",2.267544,2.267544,31.082659,6.369903,26.953290,26.858007,ele-V9,PU0,HMvDR,GENEtaBC,Em,28.542836,26.603899,28.542836
147735,23.623173,"[0.0777303799987, 2.99949836731, 5.25537061691...",-1.894929,1.894929,91.581436,26.926203,79.695659,79.617929,ele-V9,PU0,HMvDR,GENEtaBC,Em,83.094493,85.617133,83.094493
147736,31.489098,"[0.240713432431, 5.29911279678, 13.7976922989,...",-2.209277,2.209277,148.637558,32.246235,145.166468,144.925755,ele-V9,PU0,HMvDR,GENEtaBC,Em,152.521209,145.649498,152.521209
147737,87.224854,"[0.0, 2.34960699081, 14.6975412369, 44.3800735...",2.386547,2.386547,486.873138,88.782135,478.407465,478.407465,ele-V9,PU0,HMvDR,GENEtaBC,Em,483.117823,476.156237,483.117823
147738,78.769875,"[0.190564796329, 10.1607999802, 32.2133598328,...",1.875911,1.875911,265.762390,79.569847,263.115332,262.924767,ele-V9,PU0,HMvDR,GENEtaBC,Em,274.669457,269.243128,274.669457
147739,12.489261,"[0.135401308537, 3.14947319031, 7.13630628586,...",-1.924889,1.924889,70.597046,20.169779,43.715611,43.580210,ele-V9,PU0,HMvDR,GENEtaBC,Em,47.852841,49.076612,47.852841
147740,75.284500,"[0.43127822876, 10.0483188629, 26.3705883026, ...",-2.328539,2.328539,486.232880,93.863968,388.616345,388.185067,ele-V9,PU0,HMvDR,GENEtaBC,Em,407.270809,386.907334,407.270809
147741,55.441338,"[0.305906653404, 11.0419025421, 29.4075813293,...",-1.801000,1.801000,182.924820,58.810390,172.433364,172.127457,ele-V9,PU0,HMvDR,GENEtaBC,Em,188.483516,179.702992,188.483516
147742,52.842434,"[0.0, 4.99916362762, 24.620880127, 46.80467224...",2.314880,2.314880,263.317505,51.517307,270.129819,270.129819,ele-V9,PU0,HMvDR,GENEtaBC,Em,276.942120,269.081311,276.942120
147743,46.731613,"[0.0, 3.44317388535, 18.8968391418, 39.8308372...",2.171803,2.171803,214.692398,48.310295,207.677757,207.677757,ele-V9,PU0,HMvDR,GENEtaBC,Em,211.920359,209.030403,211.920359


### Fill histograms

In [48]:
#compute resolutions
# eta_uncalib_energy_nl0
# eta_lcalib_energy
tree_data['uncalib_res'] = tree_data.uncalib_energy_nl0 - tree_data.gen_energy
tree_data['lcalib_res'] = tree_data.lcalib_energy - tree_data.gen_energy
tree_data['eta_uncalib_res'] = tree_data.eta_uncalib_energy_nl0 - tree_data.gen_energy
tree_data['eta_lcalib_res'] = tree_data.eta_lcalib_energy - tree_data.gen_energy


In [49]:
hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)

for smp in samples:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    calib_table = calib_table_by_sample[smp.type]
    for tp in tps:
        for gen_sel in ['GENEtaBC', 'GENEtaD', 'GENEtaBCD']:
            for tp_sel in ['Em']:
                print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
                hres = EnergyRes(name='{}_{}_{}_{}_{}'.format(smp.type, smp.label, tp, tp_sel, gen_sel))
    
                sel_df = tree_data[(tree_data.smp == smp.type) &
                   (tree_data.pu == smp.label) &
                   (tree_data.tp == tp) &
                   (tree_data.tp_sel == tp_sel) &
                   (tree_data.gen_sel == gen_sel)]

                
                
                hres.fill_df(sel_df)

                hplot.data = hplot.data.append({'sample': smp.type,
                                       'pu': smp.label,
                                       'tp': tp,
                                       'tp_sel': tp_sel,
                                       'gen_sel': gen_sel,
                                       'classtype': EnergyRes,
                                       'histo': HWrapper(hres),},
                                       ignore_index=True)
   


------ SAMPLE: photons-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU0
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD

------ SAMPLE: ele-V9, PU: PU200
TP: HMvDR sel: Em, gen_sel: GENEtaBC
TP: HMvDR sel: Em, gen_sel: GENEtaD
TP: HMvDR sel: Em, gen_sel: GENEtaBCD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaD
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBCD


In [50]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,photons-V9,PU0,HMvDR,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29790>
1,photons-V9,PU0,HMvDR,Em,GENEtaD,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29690>
2,photons-V9,PU0,HMvDR,Em,GENEtaBCD,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29710>
3,photons-V9,PU0,HMvDRshapeDr,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd296d0>
4,photons-V9,PU0,HMvDRshapeDr,Em,GENEtaD,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29b10>
5,photons-V9,PU0,HMvDRshapeDr,Em,GENEtaBCD,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29b50>
6,ele-V9,PU0,HMvDR,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29c10>
7,ele-V9,PU0,HMvDR,Em,GENEtaD,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29750>
8,ele-V9,PU0,HMvDR,Em,GENEtaBCD,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29a50>
9,ele-V9,PU0,HMvDRshapeDr,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x11fd29f90>


In [51]:
hplot.data.tp.unique()

array(['HMvDR', 'HMvDRshapeDr'], dtype=object)

In [33]:
hplot.labels_dict

{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDHBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaBCDLBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5',
 'GENEtaBCHBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} >= 0.5',
 'GENEtaBCLBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} < 0.5',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaDHBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaDHBremPt10': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=10GeV',
 'GENEtaDHBremPt20': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=20GeV',
 'GENEtaDHBremPt30': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=30GeV',
 'GENEtaDHBremPt40': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=40GeV',
 'GENEtaDLBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5'

In [34]:
hfd = hplot.get_histo('pippo', 'ele-V9', ['PU0'], ['HMvDRcylind2p5'], 'Em', 'GENEtaBC')

print hfd[0]
print hfd[1]
print hfd[2]




No match found for: pu: ['PU0'], tp: ['HMvDRcylind2p5'], tp_sel: Em, gen_sel: GENEtaBC, classtype: pippo
None
None
None


In [35]:
print hfd[2]


None


## Comparison of the calibration sets for different selections

In [ ]:
sample='photons-V9'

In [ ]:
for tp in tps:
#     for tp_sel in tp_select[tp]:
        for gen_sel in ['GENEtaBC']:
            hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all', 'Em'], gen_sel)
            draw(hsets, labels,  text=text, min_y=0., max_y=2.)

In [ ]:
for tp in tps:
#     for tp_sel in tp_select[tp]:
#         for gen_sel in ['GENEtaBC']:
            hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['Em'], ['GENEtaBC', 'GENEtaD', 'GENEtaBCD'])
            draw(hsets, labels,  text=text, min_y=0., max_y=2.)

In [ ]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5', 'HMvDRshape', 'HMvDRshapeDr'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=10., h_lines=[1.])

In [ ]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind5', 'HMvDRcylind5Calib','HMvDRcylind5Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind10', 'HMvDRcylind10Calib','HMvDRcylind10Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRshape', 'HMvDRshapeCalib','HMvDRshapeCalib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind2p5Calib','HMvDRcylind2p5Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'Em', 'GENEtaBCPt10')
draw(hsets, labels,  text=text, min_y=0., max_y=10., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'Em', 'GENEtaBCPt40')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GEN')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    for gen_sel in ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40']:
        hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all', 'Em'], gen_sel)
        draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['Em'], ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40'])
    draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all'], ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40'])
    draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind10'], ['all', 'Em'], ['GENEtaBC'])
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
len([1.527]+[1.]*12+[1.98])

## Energy Resolution

In [41]:
def gaussfit(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()

#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', "gaus", -100, 100, 5)
#     fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y,max_value, rms_value)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, int = {}, max_value = {}, RMS = {}'.format(name, project_hist.Integral(), max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result

def gausstailfit(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, -100, 100, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result


def gausstailfit_wc(name, project_hist, bin_limits):
    global cache
    if cache is not None and not cache[(cache.h_name == name) & (cache.bin_limits == bin_limits)].empty:
        print 'READ cached fit results h_name: {}, bin_limits: {}'.format(name, bin_limits)
        print cache[(cache.h_name == name) & (cache.bin_limits == bin_limits)].results
        return cache[(cache.h_name == name) & (cache.bin_limits == bin_limits)].results.values[0]

    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, -100, 100, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    
    if cache is not None:
        
#         cache_df = pd.DataFrame(columns=['h_name', 'bin_limits', 'results'])
        cache = cache.append({'h_name': name,
                              'bin_limits': bin_limits,
                              'results': result}, ignore_index=True)
#         print cache
        
    return result




def gausstailfit_pt(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, 0, 2, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result



In [42]:
def projectAndFit(h_name, y_proj, bin_limits, cache=None):
    if cache is not None and not cache[cache.h_name == h_name].empty:
        return cache[(cache.h_name == h_name) & (cache.bin_limits == bin_limits)].results.values[0]
    

def computeEResolution(h2d_orig, cache=None):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
#         fit_result = gausstailfit(h2d_orig.GetName(), y_proj)
        fit_result = gausstailfit_wc(h2d_orig.GetName(), y_proj, (x_bin_low, x_bin_high))
        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[2]/x_mean)
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h

def computeEResolutionMean(h2d_orig, cache=None):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
#         fit_result = gausstailfit(h2d_orig.GetName(), y_proj)
        fit_result = gausstailfit_wc(h2d_orig.GetName(), y_proj, (x_bin_low, x_bin_high))

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[1])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h



### Plot response plots

In [112]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaBC')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyRes)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_etaEnergyRes)
labels_new.append('eta-calib backport')                                             

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDR'], 'Em', 'GENEtaBC')
hsets_new.append(hsets[0].h_energyRes)
labels_new.append('un-calib L1 menu')                                             

draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_559c73") at 0x7f93e66cdf00>

In [52]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaD')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyRes)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_etaEnergyRes)
labels_new.append('eta-calib backport')                                             

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDR'], 'Em', 'GENEtaD')
hsets_new.append(hsets[0].h_energyRes)
labels_new.append('un-calib L1 menu')                                             

draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_611e23") at 0x7fdf0b20cae0>

In [43]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaBC')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_etaEnergyResp)
labels_new.append('eta-calib backport')                                             

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDR'], 'Em', 'GENEtaBC')
hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib L1 menu')                                             



draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_569aca") at 0x7f93e626a970>

In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaBCD')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_etaEnergyResp)
labels_new.append('eta-calib backport')                                             

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDR'], 'Em', 'GENEtaBC')
hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib L1 menu')                                             



draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


In [44]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaD')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_etaEnergyResp)
labels_new.append('eta-calib')                                             




draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_76d401") at 0x7f93e5c7c480>

In [45]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaBC')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_lCalibEnergyResp)
labels_new.append('layer-weights')                                             

hsets_new.append(hsets[0].h_etaEnergyResp)
labels_new.append('eta-corr')                                             

hsets_new.append(hsets[0].h_etalCalibEnergyResp)
labels_new.append('layer-weights+eta-corr')                                             



draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_90bf5e") at 0x7f93e61692a0>

In [46]:

hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9'], ['PU200'], ['HMvDRshapeDr'], 'Em', 'GENEtaBCD')
hsets_new = []
labels_new = []


hsets_new.append(hsets[0].h_energyResp)
labels_new.append('un-calib')                                             

hsets_new.append(hsets[0].h_lCalibEnergyResp)
labels_new.append('layer-weights')                                             

hsets_new.append(hsets[0].h_etaEnergyResp)
labels_new.append('eta-corr')                                             

hsets_new.append(hsets[0].h_etalCalibEnergyResp)
labels_new.append('layer-weights+eta-corr')                                             



draw(hsets_new, labels_new,  text=text, options='hist', norm=True)


<ROOT.TCanvas object ("c_fbf1e1") at 0x7f93e626cea0>

### Fit resolution plots

In [47]:
# run this cell to clean the cache of the fit results
cache = pd.DataFrame(columns=['h_name', 'bin_limits', 'results'])


#### EtaBC fits

In [48]:


for sample in ['photons-V9']:
    for tp in tps:
        for pu in ['PU0']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_energyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_energyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 217.0, max_value = -4.5, RMS = 7.74357592363
   norm = 233.347074621, reso_mean = -5.02054241837, reso_sigma = 2.92406246506, alpha = 0.336247920756, n = 1916276.83933
x_low: 60.0 x_high: 120.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 295.0, max_value = -5.5, RMS = 12.1755987055
   norm = 271.632121822, reso_mean = -4.47356557843, reso_sigma = 3.72378030138, alpha = 0.323398903031, n = 54.0202207817
x_low: 120.0 x_high: 230.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 422.0, max_value = -3.5, RMS = 14.2052272769
   norm = 427.949871696, reso_mean = -5.19058405943, reso_sigma = 5.77392837933, alpha = 0.60547739145, n = 6.06684540715
x_low: 230.0 x_high: 340.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 276.0, max_value = -4.5, RMS = 13.3210576737
   norm = 265.564517297, reso_mean = -4.98965458952

In [49]:
cache

,h_name,bin_limits,results
0,photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy,"(3, 6)",<ROOT.TFitResultPtr object at 0x7f93e6361f10>
1,photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy,"(7, 12)",<ROOT.TFitResultPtr object at 0x7f93e63f0b10>
2,photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy,"(13, 23)",<ROOT.TFitResultPtr object at 0x7f93f3a43430>
3,photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy,"(24, 34)",<ROOT.TFitResultPtr object at 0x7f93e6378cf0>
4,photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy,"(35, 49)",<ROOT.TFitResultPtr object at 0x7f93e61d1270>
5,photons-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy,"(50, 100)",<ROOT.TFitResultPtr object at 0x7f93e5c176a0>
6,photons-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_energy...,"(3, 6)",<ROOT.TFitResultPtr object at 0x7f93e640c140>
7,photons-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_energy...,"(7, 12)",<ROOT.TFitResultPtr object at 0x7f93e61547d0>
8,photons-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_energy...,"(13, 23)",<ROOT.TFitResultPtr object at 0x7f93e62ba960>
9,photons-V9_PU0_HMvDRshapeDr_Em_GENEtaBC_energy...,"(24, 34)",<ROOT.TFitResultPtr object at 0x7f93e62be320>


In [50]:
for sample in ['photons-V9']:
    for tp in tps:
        for pu in ['PU0']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_lCalibEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_lCalibEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 194.0, max_value = -0.5, RMS = 8.26441277083
   norm = 201.584887256, reso_mean = -2.82309475702, reso_sigma = 3.96548810645, alpha = 0.438334056743, n = 1808381.36102
x_low: 60.0 x_high: 120.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 275.0, max_value = 0.5, RMS = 12.6570334151
   norm = 242.693496268, reso_mean = 0.300592568258, reso_sigma = 4.63917725525, alpha = 0.376236786833, n = 462963.156091
x_low: 120.0 x_high: 230.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 408.0, max_value = 2.5, RMS = 14.4221512424
   norm = 379.715096949, reso_mean = 1.59367295897, reso_sigma = 7.2358647137, alpha = 0.76259007418, n = 6.15935425416
x_low: 230.0 x_high: 340.0
   name: photons-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 247.0, max_value = 7.5, RMS = 13.8782447712
   norm = 240.340391875, reso_me

In [51]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_lCalibEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_lCalibEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 194.0, max_value = -0.5, RMS = 8.26441277083
   norm = 201.584887256, reso_mean = -2.82309475702, reso_sigma = 3.96548810645, alpha = 0.438334056743, n = 1808381.36102
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 275.0, max_value = 0.5, RMS = 12.6570334151
   norm = 242.693496268, reso_mean = 0.300592568258, reso_sigma = 4.63917725525, alpha = 0.376236786833, n = 462963.156091
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 408.0, max_value = 2.5, RMS = 14.4221512424
   norm = 379.715096949, reso_mean = 1.59367295897, reso_sigma = 7.2358647137, alpha = 0.76259007418, n = 6.15935425416
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 247.0, max_value = 7.5, RMS = 13.8782447712
   norm = 240.340391875, reso_mean = 4.054473965

   norm = 108.477477435, reso_mean = -3.13509759326, reso_sigma = 7.81190933374, alpha = 0.845080456299, n = 3875091.62522
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 203.0, max_value = 4.5, RMS = 15.3853314884
   norm = 167.653292743, reso_mean = 2.58593382536, reso_sigma = 7.81722773544, alpha = 0.647539234002, n = 11.0264609457
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 343.0, max_value = 6.5, RMS = 15.7880446184
   norm = 301.202892105, reso_mean = 5.72194262593, reso_sigma = 9.23552267222, alpha = 0.909005246924, n = 5.58371376967
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_lCalibEnergyResVenergy, y_max = 227.0, max_value = 9.5, RMS = 15.8084282811
   norm = 189.287341203, reso_mean = 9.23588504808, reso_sigma = 11.5737900242, alpha = 1.21752876352, n = 5.56214828662
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU200_HMvDRshapeDr_E

In [52]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_etaEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_etaEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etaEnergyResVenergy, y_max = 148.0, max_value = -3.5, RMS = 8.56299460352
   norm = 164.16475024, reso_mean = -2.60289534085, reso_sigma = 6.53033194124, alpha = 0.887866196825, n = 1869880.3948
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etaEnergyResVenergy, y_max = 203.0, max_value = 0.5, RMS = 12.6344744632
   norm = 217.864032924, reso_mean = 0.539107981128, reso_sigma = 5.78149682825, alpha = 0.457860054705, n = 3656969.26416
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etaEnergyResVenergy, y_max = 329.0, max_value = 1.5, RMS = 14.7158537763
   norm = 345.333390914, reso_mean = -0.198405832427, reso_sigma = 7.84154407556, alpha = 0.673527447811, n = 13.3028347957
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etaEnergyResVenergy, y_max = 228.0, max_value = 1.5, RMS = 13.9670014841
   norm = 231.10128754, reso_mean = -1.20260513316, reso_s

   norm = 144.586840651, reso_mean = 6.46081794635, reso_sigma = 4.5724782268, alpha = 0.464424629279, n = 5883196.87482
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_etaEnergyResVenergy, y_max = 235.0, max_value = 8.5, RMS = 14.1291774698
   norm = 215.161347166, reso_mean = 7.33756382127, reso_sigma = 5.19754842224, alpha = 0.477097323408, n = 10.0162392443
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_etaEnergyResVenergy, y_max = 394.0, max_value = 7.5, RMS = 14.9219979121
   norm = 367.454530109, reso_mean = 6.33746018838, reso_sigma = 6.75467264023, alpha = 0.666811332138, n = 6.54973922957
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_etaEnergyResVenergy, y_max = 244.0, max_value = 5.5, RMS = 14.4392266025
   norm = 226.9202463, reso_mean = 5.01778473705, reso_sigma = 8.95527638556, alpha = 0.860971283191, n = 9.84532597104
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_e

In [45]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_energyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_energyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 217.0, max_value = -4.5, RMS = 7.74357592363
   norm = 233.347412896, reso_mean = -5.02057911268, reso_sigma = 2.92409361291, alpha = 0.336254088121, n = 3054672.78738
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 295.0, max_value = -5.5, RMS = 12.1755987055
   norm = 271.632121822, reso_mean = -4.47356557843, reso_sigma = 3.72378030138, alpha = 0.323398903031, n = 54.0202207817
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 422.0, max_value = -3.5, RMS = 14.2052272769
   norm = 427.949871696, reso_mean = -5.19058405943, reso_sigma = 5.77392837933, alpha = 0.60547739145, n = 6.06684540715
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 276.0, max_value = -4.5, RMS = 13.3210576737
   norm = 265.564517297, reso_mean = -4.98965458952, reso_sigma = 7

   norm = 130.37643427, reso_mean = -5.62371242191, reso_sigma = 5.8096241111, alpha = 0.617243684132, n = 1688123.95171
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_energyResVenergy, y_max = 236.0, max_value = -4.5, RMS = 14.354564443
   norm = 191.618073647, reso_mean = -3.92772505503, reso_sigma = 6.36536179193, alpha = 0.556413613627, n = 10.9461439022
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_energyResVenergy, y_max = 382.0, max_value = -5.5, RMS = 15.3866961193
   norm = 341.671274124, reso_mean = -4.59126591929, reso_sigma = 7.7086879666, alpha = 0.822561638619, n = 4.3777277875
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_energyResVenergy, y_max = 233.0, max_value = -2.5, RMS = 14.7320624211
   norm = 214.130375751, reso_mean = -4.55742420052, reso_sigma = 9.77850348781, alpha = 0.979519751675, n = 7.38001809223
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_energ

In [53]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
    #         print dir(hsets[0])
            draw([his.h_etalCalibEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_etalCalibEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etalCalibEnergyResVenergy, y_max = 194.0, max_value = -0.5, RMS = 8.26441277083
   norm = 201.584887256, reso_mean = -2.82309475702, reso_sigma = 3.96548810645, alpha = 0.438334056743, n = 1808381.36102
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etalCalibEnergyResVenergy, y_max = 275.0, max_value = 0.5, RMS = 12.6570334151
   norm = 242.693496268, reso_mean = 0.300592568258, reso_sigma = 4.63917725525, alpha = 0.376236786833, n = 462963.156091
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etalCalibEnergyResVenergy, y_max = 408.0, max_value = 2.5, RMS = 14.4221512424
   norm = 379.715096949, reso_mean = 1.59367295897, reso_sigma = 7.2358647137, alpha = 0.76259007418, n = 6.15935425416
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaBC_etalCalibEnergyResVenergy, y_max = 247.0, max_value = 7.5, RMS = 13.8782447712
   norm = 240.340391875, reso_mean =

   norm = 108.477477435, reso_mean = -3.13509759326, reso_sigma = 7.81190933374, alpha = 0.845080456299, n = 3875091.62522
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_etalCalibEnergyResVenergy, y_max = 203.0, max_value = 4.5, RMS = 15.3853314884
   norm = 167.653292743, reso_mean = 2.58593382536, reso_sigma = 7.81722773544, alpha = 0.647539234002, n = 11.0264609457
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_etalCalibEnergyResVenergy, y_max = 343.0, max_value = 6.5, RMS = 15.7880446184
   norm = 301.202892105, reso_mean = 5.72194262593, reso_sigma = 9.23552267222, alpha = 0.909005246924, n = 5.58371376967
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaBC_etalCalibEnergyResVenergy, y_max = 227.0, max_value = 9.5, RMS = 15.8084282811
   norm = 189.287341203, reso_mean = 9.23588504808, reso_sigma = 11.5737900242, alpha = 1.21752876352, n = 5.56214828662
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU200_HMvDR

#### EtaD filts

In [54]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaD', debug=False)
    #         print dir(hsets[0])
            draw([his.h_energyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_energyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_energyResVenergy, y_max = 56.0, max_value = -4.5, RMS = 8.27250872458
   norm = 53.5329168841, reso_mean = -5.8917561698, reso_sigma = 2.33623444901, alpha = 0.263936845963, n = 2104686.0037
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_energyResVenergy, y_max = 60.0, max_value = -10.5, RMS = 14.2645266557
   norm = 48.0815388672, reso_mean = -8.15769190631, reso_sigma = 4.33671063275, alpha = 0.280384076783, n = 2648603.71785
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_energyResVenergy, y_max = 76.0, max_value = -8.5, RMS = 22.8082708565
   norm = 74.9736525811, reso_mean = -9.99339493098, reso_sigma = 6.16489985549, alpha = 0.797227825789, n = 1.110915539
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_energyResVenergy, y_max = 65.0, max_value = -13.5, RMS = 25.1586895347
   norm = 27.5669386388, reso_mean = -26.0086342002, reso_sigma = 25.3703949

   norm = 7.67568110987, reso_mean = -2.29505052632, reso_sigma = 11.998152529, alpha = 1.45428514114, n = 410114.185134
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_energyResVenergy, y_max = 21.0, max_value = -0.5, RMS = 20.7003177234
   norm = 14.9755477985, reso_mean = 1.40276962334, reso_sigma = 11.1835935345, alpha = 0.516604436403, n = 2118963.61321
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_energyResVenergy, y_max = 48.0, max_value = 2.5, RMS = 27.3814444596
   norm = 33.187896839, reso_mean = 4.58390095949, reso_sigma = 12.7119565243, alpha = 0.673771136785, n = 2.21670243472
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_energyResVenergy, y_max = 41.0, max_value = 3.5, RMS = 31.9733626845
   norm = 31.3099252059, reso_mean = 5.06007917056, reso_sigma = 14.9653279843, alpha = 0.9561347388, n = 0.79990940562
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_energyResVenergy,

In [55]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaD', debug=False)
    #         print dir(hsets[0])
            draw([his.h_etaEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_etaEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_etaEnergyResVenergy, y_max = 44.0, max_value = -13.5, RMS = 8.66813636724
   norm = 44.1421956667, reso_mean = -12.4632026992, reso_sigma = 3.74488934246, alpha = 0.40646206946, n = 2204867.11709
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_etaEnergyResVenergy, y_max = 51.0, max_value = -17.5, RMS = 14.7098307296
   norm = 44.6155450658, reso_mean = -13.8933426857, reso_sigma = 5.02263836321, alpha = 0.311139030909, n = 5505787.45811
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_etaEnergyResVenergy, y_max = 71.0, max_value = -11.5, RMS = 22.3410627841
   norm = 69.9217531584, reso_mean = -15.6792842115, reso_sigma = 6.91226924347, alpha = 0.913276701694, n = 0.927713491009
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_etaEnergyResVenergy, y_max = 57.0, max_value = -17.5, RMS = 23.8207375245
   norm = 52.2980838875, reso_mean = -17.2191980874, reso_

   norm = 7.71183548497, reso_mean = -0.190274483001, reso_sigma = 11.9125084175, alpha = 1.4178691214, n = 3511719.1223
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_etaEnergyResVenergy, y_max = 27.0, max_value = 3.5, RMS = 20.927105229
   norm = 14.9930389151, reso_mean = 4.00796158184, reso_sigma = 10.8714917841, alpha = 0.489346870935, n = 2360665.5662
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_etaEnergyResVenergy, y_max = 50.0, max_value = 6.5, RMS = 27.7725913586
   norm = 21.3455380765, reso_mean = -7.86432258194, reso_sigma = 27.8623781145, alpha = -515752.114913, n = 33.0332342727
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_etaEnergyResVenergy, y_max = 40.0, max_value = 5.5, RMS = 32.6007522155
   norm = 31.9979290934, reso_mean = 7.84578693712, reso_sigma = 14.2371316613, alpha = 0.922772257696, n = 0.812755578286
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_etaEne

In [56]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaD', debug=False)
    #         print dir(hsets[0])
            draw([his.h_lCalibEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_lCalibEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_lCalibEnergyResVenergy, y_max = 47.0, max_value = -3.5, RMS = 8.8809778834
   norm = 45.046556399, reso_mean = -3.87149396668, reso_sigma = 3.49061498596, alpha = 0.373551080496, n = 2325894.09901
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_lCalibEnergyResVenergy, y_max = 54.0, max_value = -5.5, RMS = 14.9490516169
   norm = 43.7903691779, reso_mean = -4.16147350849, reso_sigma = 5.41510876932, alpha = 0.340404162392, n = 1641452.90005
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_lCalibEnergyResVenergy, y_max = 76.0, max_value = -4.5, RMS = 24.2311563583
   norm = 68.5944690257, reso_mean = -2.93128672282, reso_sigma = 6.91129665118, alpha = 0.775929274835, n = 1.2187855218
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_lCalibEnergyResVenergy, y_max = 63.0, max_value = -3.5, RMS = 27.2589476738
   norm = 51.907861979, reso_mean = -2.75808940903, r

   norm = 6.44484740263, reso_mean = 2.27542886115, reso_sigma = 14.6085878842, alpha = 5.30538163079, n = 6.06823616813
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_lCalibEnergyResVenergy, y_max = 19.0, max_value = 7.5, RMS = 22.7099185519
   norm = 13.1738726819, reso_mean = 9.41952213362, reso_sigma = 13.3783489562, alpha = 0.576052911704, n = 1233575.56566
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_lCalibEnergyResVenergy, y_max = 46.0, max_value = 15.5, RMS = 29.8301381261
   norm = 29.2892203257, reso_mean = 16.4282766323, reso_sigma = 15.1555097859, alpha = 0.69723047737, n = 2.77223243459
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_lCalibEnergyResVenergy, y_max = 37.0, max_value = 19.5, RMS = 36.1158758194
   norm = 27.4638212483, reso_mean = 20.865297971, reso_sigma = 17.9038672253, alpha = 0.981596390344, n = 0.862902846472
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENE

In [57]:
for sample in ['ele-V9']:
    for tp in tps:
        for pu in ['PU0', 'PU200']:
            hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaD', debug=False)
    #         print dir(hsets[0])
            draw([his.h_etalCalibEnergyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
            draw([his.h_etalCalibEnergyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_etalCalibEnergyResVenergy, y_max = 47.0, max_value = -3.5, RMS = 8.8809778834
   norm = 45.046556399, reso_mean = -3.87149396668, reso_sigma = 3.49061498596, alpha = 0.373551080496, n = 2325894.09901
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_etalCalibEnergyResVenergy, y_max = 54.0, max_value = -5.5, RMS = 14.9490516169
   norm = 43.7903691779, reso_mean = -4.16147350849, reso_sigma = 5.41510876932, alpha = 0.340404162392, n = 1641452.90005
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_etalCalibEnergyResVenergy, y_max = 76.0, max_value = -4.5, RMS = 24.2311563583
   norm = 68.5944690257, reso_mean = -2.93128672282, reso_sigma = 6.91129665118, alpha = 0.775929274835, n = 1.2187855218
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU0_HMvDR_Em_GENEtaD_etalCalibEnergyResVenergy, y_max = 63.0, max_value = -3.5, RMS = 27.2589476738
   norm = 51.907861979, reso_mean = -2.75

   norm = 6.44484740263, reso_mean = 2.27542886115, reso_sigma = 14.6085878842, alpha = 5.30538163079, n = 6.06823616813
x_low: 60.0 x_high: 120.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_etalCalibEnergyResVenergy, y_max = 19.0, max_value = 7.5, RMS = 22.7099185519
   norm = 13.1738726819, reso_mean = 9.41952213362, reso_sigma = 13.3783489562, alpha = 0.576052911704, n = 1233575.56566
x_low: 120.0 x_high: 230.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_etalCalibEnergyResVenergy, y_max = 46.0, max_value = 15.5, RMS = 29.8301381261
   norm = 29.2892203257, reso_mean = 16.4282766323, reso_sigma = 15.1555097859, alpha = 0.69723047737, n = 2.77223243459
x_low: 230.0 x_high: 340.0
   name: ele-V9_PU200_HMvDRshapeDr_Em_GENEtaD_etalCalibEnergyResVenergy, y_max = 37.0, max_value = 19.5, RMS = 36.1158758194
   norm = 27.4638212483, reso_mean = 20.865297971, reso_sigma = 17.9038672253, alpha = 0.981596390344, n = 0.862902846472
x_low: 340.0 x_high: 490.0
   name: ele-V9_PU200_HMvDRshapeD

### Draw resolution fit results

In [67]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


AttributeError: EnergyRes instance has no attribute 'g_energyResVenergy_meanEReso'

In [59]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


AttributeError: EnergyRes instance has no attribute 'g_energyResVenergy_meanEReso'

In [60]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_etaEnergyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [113]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.h_etaEnergyResVenergy for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [61]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_etaEnergyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


In [62]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], 'PU0', ['HMvDRCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


No match found for: pu: PU0, tp: ['HMvDRCalib'], tp_sel: ['Em'], gen_sel: GENEtaBC, classtype: __main__.EnergyRes


TypeError: 'NoneType' object is not iterable

In [63]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], 'PU0', ['HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-50, max_y=50)


No match found for: pu: PU0, tp: ['HMvDRshapeDrCalib'], tp_sel: ['Em'], gen_sel: GENEtaBC, classtype: __main__.EnergyRes


TypeError: 'NoneType' object is not iterable

In [64]:
for sample in ['ele-V9', 'ele-V10']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


AttributeError: EnergyRes instance has no attribute 'g_energyResVenergy_sigmaEOE'

In [68]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


AttributeError: EnergyRes instance has no attribute 'g_energyResVenergy_sigmaEOE'

In [ ]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_etaEnergyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR','HMvDRCalib', 'HMvDRshapeDr', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_etaEnergyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [69]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
    hset_new = []
    labels_new = []

    hset_new.append(hsets[0].g_energyResVenergy_sigmaEOE)
    labels_new.append('no corr.')

    hset_new.append(hsets[0].g_etaEnergyResVenergy_sigmaEOE)
    labels_new.append('eta corr.')

    hset_new.append(hsets[0].g_lCalibEnergyResVenergy_sigmaEOE)
    labels_new.append('layer calib.')

    hset_new.append(hsets[0].g_etalCalibEnergyResVenergy_sigmaEOE)
    labels_new.append('eta+ layer calib.')

    
    draw(hset_new, labels_new, options='L', text=text,  min_y=-0, max_y=0.1)


AttributeError: EnergyRes instance has no attribute 'g_energyResVenergy_sigmaEOE'

In [83]:
for sample in ['ele-V9']:
#     hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDR'], ['Em'], 'GENEtaBC', debug=False)
#     hset_new = []
#     labels_new = []

#     hset_new.append(hsets[0].g_energyResVenergy_sigmaEOE)
#     labels_new.append('L1 menu')

    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU200', ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)

    hset_new.append(hsets[0].g_energyResVenergy_sigmaEOE)
    labels_new.append('backport EM interp.')

    hset_new.append(hsets[0].g_etaEnergyResVenergy_sigmaEOE)
    labels_new.append('eta corr.')

#     hset_new.append(hsets[0].g_lCalibEnergyResVenergy_sigmaEOE)
#     labels_new.append('layer calib.')

#     hset_new.append(hsets[0].g_etalCalibEnergyResVenergy_sigmaEOE)
#     labels_new.append('eta+ layer calib.')

    
    draw(hset_new, labels_new, options='L', text=text,  min_y=-0, max_y=0.2)


AttributeError: EnergyRes instance has no attribute 'g_energyResVenergy_sigmaEOE'

In [71]:
for sample in ['photons-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
    hset_new = []
    labels_new = []

    hset_new.append(hsets[0].g_energyResVenergy_meanEReso)
    labels_new.append('no corr.')

    hset_new.append(hsets[0].g_lCalibEnergyResVenergy_meanEReso)
    labels_new.append('layer calib.')

    
    draw(hset_new, labels_new, options='L', text=text,  min_y=-10, max_y=10)


In [72]:
for sample in ['photons-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
    hset_new = []
    labels_new = []

    hset_new.append(hsets[0].g_energyResVenergy_sigmaEOE)
    labels_new.append('no corr.')

    hset_new.append(hsets[0].g_lCalibEnergyResVenergy_sigmaEOE)
    labels_new.append('layer calib.')

    
    draw(hset_new, labels_new, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
for sample in ['ele-V9']:
    hsets, labels, text = hplot.get_histo(EnergyRes, sample,[ 'PU0', 'PU200'], ['HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
    draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], 'PU0', ['HMvDRCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, ['ele-V9', 'ele-V10'], 'PU0', ['HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, options='L', text=text,  min_y=-0, max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshape','HMvDRcylind2p5','HMvDRcylind5', 'HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeCalib','HMvDRcylind2p5Calib','HMvDRcylind5Calib', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeCalib1','HMvDRcylind2p5Calib1','HMvDRcylind5Calib1', 'HMvDRcylind10Calib1'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
    

In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRfixedCalib1','HMvDRfixed'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRcylind2p5','HMvDRcylind2p5Calib1'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
